# 04 - Supervised Learning (Classification)

This notebook includes:
- Target variable definition (pass/fail, risk levels)
- Feature engineering and encoding
- Multiple classification algorithms
- Hyperparameter tuning
- Cross-validation
- Model interpretation